<a href="https://colab.research.google.com/github/amylynnn/weatherwise-Amylynn-Sophie/blob/main/draft1FINAL__starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌦️ WeatherWise



In [ ]:
# 🧪 Optional packages — uncomment if needed in Colab or JupyterHub
#import requests # For making HTTP requests to fetch data from the web or APIs
#import matplotlib.pyplot as plt # For creating static, animated, and interactive visualizations in Python
#import ipywidgets as widgets # For adding interactive widgets (like sliders, buttons) in Jupyter notebooks
#from IPython.display import display, clear_output # For displaying widgets and clearing output in Jupyter notebooks
#import re # For working with regular expressions, useful for pattern matching in strings
#from datetime import datetime, timedelta # For manipulating dates and times



In [ ]:
API_KEY = '7cf20335110caaf78db0fecb31852d45'
# unique API key used to authenticate requests weather service API.
#7cf20335110caaf78db0fecb31852d45

# Simple in-memory cache: { (source, city): (timestamp, data) }
weather_cache = {}
# A dictionary that stores cached weather data to avoid repeated API calls.
CACHE_EXPIRY_MINUTES = 10  # Cache duration

## 📦 Setup and Configuration
Import required packages and setup environment.

In [ ]:
#import requests
#import matplotlib.pyplot as plt
#import seaborn as sns
#import plotly.express as px
#import pandas as pd
#import re

# I added two more codes to enhance the visualisation
#from fetch_my_weather import get_weather
#from hands_on_ai.chat import get_response
# Typing support for cleaner, testable function signatures
#from typing import Union, List, Dict

import requests                # For HTTP requests to weather APIs
import matplotlib.pyplot as plt # For plotting weather charts
import ipywidgets as widgets    # For interactive UI elements in Jupyter/Colab
from IPython.display import display, clear_output # To display widgets and clear output areas
import re                      # For parsing user questions with regular expressions
from datetime import datetime, timedelta # For handling dates and times
from random import choice      # For picking random humorous messages

## 🌤️ Weather Data Functions

In [ ]:
# Define get_weather_data() function here
def fetch_weather_data(city, source='openweathermap'):
    """Fetch weather data for a city, using cache if available and valid."""
    # Key is (source, city) to support multiple sources and cities
    key = (source.lower(), city.lower())
    now = datetime.utcnow()
    # Use cached data if it's still fresh
    if key in weather_cache:
        timestamp, cached_data = weather_cache[key]
        if now - timestamp < timedelta(minutes=CACHE_EXPIRY_MINUTES):
            return cached_data
    # Otherwise, fetch new data
    if source.lower() == 'openweathermap':
        data = fetch_openweathermap(city)
    elif source.lower() == 'mocksource':
        data = fetch_mock_source(city)
    else:
        raise ValueError(f"Unsupported weather data source: {source}")
    weather_cache[key] = (now, data)
    return data

def fetch_openweathermap(city): #Calls the actual weather API and returns the JSON forecast.
    """Call OpenWeatherMap API to get weather forecast for the given city."""
    url = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    response.raise_for_status()
    return response.json()

    def fetch_mock_source(city):
    """Generate random mock weather data for testing/demo."""
    from random import uniform, randint
    now = datetime.utcnow()
    list_data = []
    for i in range(40):  # 5 days, 3-hour intervals
        dt = now + timedelta(hours=3*i)
        dt_txt = dt.strftime("%Y-%m-%d %H:%M:%S")
        temp = round(uniform(10, 25), 1)
        rain = round(uniform(0, 5), 1) if randint(0, 1) else 0
        snow = round(uniform(0, 2), 1) if randint(0, 3) == 0 else 0
        weather_desc = 'clear sky' if randint(0, 2) == 0 else 'rain'
        entry = {
            'dt_txt': dt_txt,
            'main': {'temp': temp},
            'rain': {'3h': rain} if rain > 0 else {},
            'snow': {'3h': snow} if snow > 0 else {},
            'weather': [{'description': weather_desc}],
            'wind': {'speed': round(uniform(0, 10),1), 'deg': randint(0,360)}
        }
        list_data.append(entry)
    return {'list': list_data} #Creates fake weather data for testing the app without hitting the API.





## 📊 Visualisation Functions

In [ ]:
# Define create_temperature_visualisation() and create_precipitation_visualisation() here
def interactive_temperature_plot(data, city): #I used interactive instead to let user interactively explore the temperature forecast for the next days
    """Show an interactive temperature line chart with a slider."""
    times = [datetime.strptime(entry['dt_txt'], "%Y-%m-%d %H:%M:%S") for entry in data['list']]
    temps = [entry['main']['temp'] for entry in data['list']]
    def plot_upto(index=8):
        plt.figure(figsize=(12, 6))
        plt.plot(times[:index], temps[:index], marker='o', linestyle='-', color='orange')
        plt.title(f"Temperature Forecast for {city.title()}")
        plt.xlabel("Date & Time")
        plt.ylabel("Temperature (°C)")
        plt.xticks(rotation=45)
        plt.grid(True)
        plt.tight_layout()
        plt.show()
    widgets.interact(plot_upto, index=widgets.IntSlider(min=1, max=len(times), step=1, value=8))




In [ ]:
def interactive_precipitation_plot(data, city): #I used interactive instead to lets user interactively explore the rain forecast for the next days.
    """Show an interactive precipitation bar chart with a slider."""
    times = [datetime.strptime(entry['dt_txt'], "%Y-%m-%d %H:%M:%S") for entry in data['list']]
    rain = [entry.get('rain', {}).get('3h', 0) for entry in data['list']]
    def plot_upto(index=8):
        plt.figure(figsize=(12, 6))
        plt.bar(times[:index], rain[:index], width=0.1, color='blue', edgecolor='black')
        plt.title(f"Precipitation Forecast for {city.title()} (Next 5 Days)")
        plt.xlabel("Date & Time")
        plt.ylabel("Rain (mm in 3h)")
        plt.xticks(rotation=45)
        plt.grid(axis='y', linestyle='--', alpha=0.5)
        plt.tight_layout()
        plt.show()
    widgets.interact(plot_upto, index=widgets.IntSlider(min=1, max=len(times), step=1, value=8))


## 🤖 Natural Language Processing

In [ ]:
#Extracts the user's intent (rain, temperature, etc.), city, and time frame from their question.
def parse_weather_question(question):
    """Parse user's question to extract weather condition, city, and time frame."""
    question_lower = question.lower()
    # Synonyms for weather conditions
    condition_keywords = {
        'rain': [...], 'snow': [...], 'clear': [...], 'cloudy': [...],
        'temperature': [...], 'wind': [...], 'humidity': [...], 'storm': [...], 'fog': [...]
    }
    # Detect which weather condition is being asked about
    condition = None
    for cond, synonyms in condition_keywords.items():
        for syn in synonyms:
            if syn in question_lower:
                condition = cond
                break
        if condition:
            break
    if not condition:
        condition = 'general'  # Default if no keyword found

    # Detect time frame (today, tomorrow, etc.)
    if 'day after tomorrow' in question_lower:
        time_frame = 'day_after_tomorrow'
    elif 'tomorrow' in question_lower:
        time_frame = 'tomorrow'
    elif 'today' in question_lower:
        time_frame = 'today'
    else:
        match = re.search(r'next (\d+) days', question_lower)
        if match:
            time_frame = f"next_{match.group(1)}_days"
        elif 'this weekend' in question_lower:
            time_frame = 'this_weekend'
        else:
            time_frame = 'today'

    # Try to extract city name after "in"
    city_match = re.search(
        r"in ([a-zA-ZÀ-ÿ\s\-]+?)(?:\s+(?:today|tomorrow|day after tomorrow|next \d+ days|this weekend)|\?|$| ... )",
        question_lower)
    if city_match:
        city = city_match.group(1).strip()
    else:
        # Fallback: last capitalized word as city
        capitalized = re.findall(r"\b[A-Z][a-z]+(?:\s[A-Z][a-z]+)*", question)
        city = capitalized[-1] if capitalized else None

    # Normalize some city names to include country code for better API results
    fixes = {...}
    if city:
        city_key = city.lower()
        city = fixes.get(city_key, city.title())
    else:
        city = None

    return {
        'condition': condition,
        'city': city,
        'time_frame': time_frame
    }


## 🧭 User Interface

In [ ]:
def interactive_ui():
    """Build and display the main weather assistant UI with all widgets and layout."""
    # Input fields for city, question, and data source selection
    city_input = widgets.Text(...)
    question_input = widgets.Text(...)
    source_dropdown = widgets.Dropdown(...)

    # Output area for weather answers and charts
    output_area = widgets.Output(...)

    # Buttons for forecast, answering, and clearing
    forecast_btn = widgets.Button(...)
    question_btn = widgets.Button(...)
    clear_btn = widgets.Button(...)

    # Enable/disable buttons based on input fields
    def toggle_buttons(*args):
        forecast_btn.disabled = not bool(city_input.value.strip())
        question_btn.disabled = not bool(question_input.value.strip())
    city_input.observe(toggle_buttons, 'value')
    question_input.observe(toggle_buttons, 'value')
    toggle_buttons()

    # Button click handlers
    def on_forecast_button_click(_):
        with output_area:
            clear_output()
            city = city_input.value.strip()
            if not city:
                print("❌ Please enter a city.")
                return
            try:
                data = fetch_weather_data(city, source=source_dropdown.value)
                interactive_temperature_plot(data, city)
                interactive_precipitation_plot(data, city)
            except Exception as e:
                print(f"❌ Error: {e}")

    def on_question_button_click(_):
        with output_area:
            clear_output()
            question = question_input.value.strip()
            if not question:
                print("❌ Please enter a question.")
                return
            parsed = parse_weather_question(question)
            response = generate_weather_response(parsed, source=source_dropdown.value)
            print(response)

    def on_clear_button_click(_):
        city_input.value = ''
        question_input.value = ''
        with output_area:
            clear_output()

    # Attach handlers to buttons
    forecast_btn.on_click(on_forecast_button_click)
    question_btn.on_click(on_question_button_click)
    clear_btn.on_click(on_clear_button_click)

    # Layout rows for inputs and output
    input_row = widgets.HBox([...])
    question_row = widgets.HBox([...])

    # Header for the app
    header = widgets.HTML(...)

    # Main box: the card-like container for all widgets
    main_box = widgets.VBox([
        header,
        input_row,
        question_row,
        output_area
    ], layout=widgets.Layout(...))

    display(main_box)

# Run the UI
interactive_ui()


## 🧩 Main Application Logic

In [ ]:
import requests
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import re
from datetime import datetime, timedelta
from random import choice

API_KEY = '7cf20335110caaf78db0fecb31852d45'
weather_cache = {}
CACHE_EXPIRY_MINUTES = 10

def fetch_weather_data(city, source='openweathermap'):
    global weather_cache
    key = (source.lower(), city.lower())
    now = datetime.utcnow()
    if key in weather_cache:
        timestamp, cached_data = weather_cache[key]
        if now - timestamp < timedelta(minutes=CACHE_EXPIRY_MINUTES):
            return cached_data
    if source.lower() == 'openweathermap':
        data = fetch_openweathermap(city)
    elif source.lower() == 'mocksource':
        data = fetch_mock_source(city)
    else:
        raise ValueError(f"Unsupported weather data source: {source}")
    weather_cache[key] = (now, data)
    return data

def fetch_openweathermap(city):
    url = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    response.raise_for_status()
    return response.json()

def fetch_mock_source(city):
    from random import uniform, randint
    now = datetime.utcnow()
    list_data = []
    for i in range(40):
        dt = now + timedelta(hours=3*i)
        dt_txt = dt.strftime("%Y-%m-%d %H:%M:%S")
        temp = round(uniform(10, 25), 1)
        rain = round(uniform(0, 5), 1) if randint(0, 1) else 0
        snow = round(uniform(0, 2), 1) if randint(0, 3) == 0 else 0
        weather_desc = 'clear sky' if randint(0, 2) == 0 else 'rain'
        entry = {
            'dt_txt': dt_txt,
            'main': {'temp': temp},
            'rain': {'3h': rain} if rain > 0 else {},
            'snow': {'3h': snow} if snow > 0 else {},
            'weather': [{'description': weather_desc}],
            'wind': {'speed': round(uniform(0, 10),1), 'deg': randint(0,360)}
        }
        list_data.append(entry)
    return {'list': list_data}

def parse_weather_question(question):
    question_lower = question.lower()

    condition_keywords = {
        'rain': ['rain', 'raining', 'rainy', 'wet', 'shower', 'showers', 'drizzle'],
        'snow': ['snow', 'snowing', 'snowy', 'flurries'],
        'clear': ['clear', 'sunny', 'bright'],
        'cloudy': ['cloud', 'cloudy', 'overcast'],
        'temperature': ['temperature', 'hot', 'cold', 'warm', 'chilly', 'freezing', 'heat'],
        'wind': ['wind', 'windy', 'breeze', 'gust'],
        'humidity': ['humidity', 'humid', 'dry'],
        'storm': ['storm', 'thunderstorm', 'lightning', 'hail'],
        'fog': ['fog', 'mist', 'haze']
    }

    condition = None
    for cond, synonyms in condition_keywords.items():
        for syn in synonyms:
            if syn in question_lower:
                condition = cond
                break
        if condition:
            break
    if not condition:
        condition = 'general'

    if 'day after tomorrow' in question_lower:
        time_frame = 'day_after_tomorrow'
    elif 'tomorrow' in question_lower:
        time_frame = 'tomorrow'
    elif 'today' in question_lower:
        time_frame = 'today'
    else:
        match = re.search(r'next (\d+) days', question_lower)
        if match:
            time_frame = f"next_{match.group(1)}_days"
        elif 'this weekend' in question_lower:
            time_frame = 'this_weekend'
        else:
            time_frame = 'today'

    city_match = re.search(
        r"in ([a-zA-ZÀ-ÿ\s\-]+?)(?:\s+(?:today|tomorrow|day after tomorrow|next \d+ days|this weekend)|\?|$| will| is| does| rain| snow| clear| cloudy| thunderstorm| drizzle| fog| mist| temperature| wind| humidity| storm)",
        question_lower)
    if city_match:
        city = city_match.group(1).strip()
    else:
        capitalized = re.findall(r"\b[A-Z][a-z]+(?:\s[A-Z][a-z]+)*", question)
        city = capitalized[-1] if capitalized else None

    fixes = {
        "paris": "Paris,FR",
        "london": "London,GB",
        "new york": "New York,US",
        "berlin": "Berlin,DE",
        "sydney": "Sydney,AU"
    }
    if city:
        city_key = city.lower()
        city = fixes.get(city_key, city.title())
    else:
        city = None

    return {
        'condition': condition,
        'city': city,
        'time_frame': time_frame
    }

def generate_weather_response(parsed, source='openweathermap'):
    if not parsed or not parsed.get('city'):
        return "⚠️ Sorry, I couldn't understand your question or detect the city. Please try again."

    condition = parsed['condition']
    city = parsed['city']
    time_frame = parsed['time_frame']

    try:
        data = fetch_weather_data(city, source=source)
        today = datetime.utcnow().date()

        if time_frame == 'tomorrow':
            target_date = today + timedelta(days=1)
        elif time_frame == 'day_after_tomorrow':
            target_date = today + timedelta(days=2)
        elif time_frame.startswith('next_'):
            days = int(time_frame.split('_')[1])
            temps = []
            rains = []
            for entry in data['list']:
                entry_date = datetime.strptime(entry['dt_txt'], "%Y-%m-%d %H:%M:%S").date()
                if today <= entry_date < today + timedelta(days=days):
                    temps.append(entry['main']['temp'])
                    rains.append(entry.get('rain', {}).get('3h', 0))
            if temps:
                avg_temp = sum(temps) / len(temps)
                total_rain = sum(rains)
                return (f"🌤️ Over the next {days} days in {city}, expect an average temperature of "
                        f"{avg_temp:.1f}°C and total precipitation around {total_rain:.1f} mm.")
            else:
                return f"⚠️ Sorry, no forecast data available for the next {days} days in {city}."
        else:
            target_date = today

        day_entries = [entry for entry in data['list'] if datetime.strptime(entry['dt_txt'], "%Y-%m-%d %H:%M:%S").date() == target_date]
        if not day_entries:
            return f"⚠️ Sorry, no forecast data available for {time_frame.replace('_', ' ')} in {city}."

        temps = [entry['main']['temp'] for entry in day_entries]
        avg_temp = sum(temps) / len(temps)
        rain_volumes = [entry.get('rain', {}).get('3h', 0) for entry in day_entries]
        total_rain = sum(rain_volumes)
        wind_speeds = [entry.get('wind', {}).get('speed', 0) for entry in day_entries]
        max_wind = max(wind_speeds) if wind_speeds else 0

        will_rain = total_rain > 1
        is_cold = avg_temp < 10
        is_hot = avg_temp > 25
        windy = max_wind > 15

        messages = []
        messages.append(f"🌤️ Weather forecast for {city} {time_frame.replace('_', ' ')}:")
        messages.append(f"Average temperature: {avg_temp:.1f}°C.")

        if will_rain:
            rain_msgs = [
                "Pack an umbrella, unless you're secretly a duck. ☔",
                "Rain is on the forecast! Time to dance in the puddles...or just stay inside with a good book. 🌧️",
                "The sky is crying today - bring a raincoat and cheer it up! 😢",
            ]
            messages.append(choice(rain_msgs))
        else:
            clear_msgs = [
                "Clear skies ahead! Time to wear those sunglasses 😎",
                "Looks like the sun's got its hat on! Don't forget the sunscreen. ☀️",
                "Not a cloud in sight - perfect day for a picnic! 🧺",
            ]
            messages.append(choice(clear_msgs))

        if is_cold:
            cold_msgs = [
                "It's colder than a penguin's toes out there! Bundle up! 🐧",
                "Time to build a snowman... or just hide under a blanket. ☃️",
                "Beware of rogue icicles! Dress warmly, or you'll turn into a human popsicle. 🥶",
            ]
            messages.append(choice(cold_msgs))
        elif is_hot:
            hot_msgs = [
                "It's hotter than a dragon's breath! Stay hydrated! 🐉",
                "Time to hit the beach...or just sit in front of the AC. 🏖️",
                "Make sure to hydrate today, or you may end up melting like the polar ice caps. 🥵",
            ]
            messages.append(choice(hot_msgs))

        if windy:
            windy_msgs = [
                "Hold onto your hats, it's gonna be a wild one! 💨",
                "Prepare for a hair-raising experience - the wind is howling! 💇‍♀️",
                "Make sure all loose items are secured, or they may end up in another country! 🌪️",
            ]
            messages.append(choice(windy_msgs))

        messages.append(f"Total precipitation expected: {total_rain:.1f} mm.")
        messages.append("Have a great day! ☀️")

        return "\n".join(messages)

    except Exception as e:
        return f"❌ Could not retrieve weather for {city}: {e}"

def interactive_temperature_plot(data, city):
    times = [datetime.strptime(entry['dt_txt'], "%Y-%m-%d %H:%M:%S") for entry in data['list']]
    temps = [entry['main']['temp'] for entry in data['list']]

    def plot_upto(index=8):
        plt.figure(figsize=(12, 6))
        plt.plot(times[:index], temps[:index], marker='o', linestyle='-', color='orange')
        plt.title(f"Temperature Forecast for {city.title()}")
        plt.xlabel("Date & Time")
        plt.ylabel("Temperature (°C)")
        plt.xticks(rotation=45)
        plt.grid(True)
        plt.tight_layout()
        plt.show()

    widgets.interact(plot_upto, index=widgets.IntSlider(min=1, max=len(times), step=1, value=8))

def interactive_precipitation_plot(data, city):
    times = [datetime.strptime(entry['dt_txt'], "%Y-%m-%d %H:%M:%S") for entry in data['list']]
    rain = [entry.get('rain', {}).get('3h', 0) for entry in data['list']]

    def plot_upto(index=8):
        plt.figure(figsize=(12, 6))
        plt.bar(times[:index], rain[:index], width=0.1, color='blue', edgecolor='black')
        plt.title(f"Precipitation Forecast for {city.title()} (Next 5 Days)")
        plt.xlabel("Date & Time")
        plt.ylabel("Rain (mm in 3h)")
        plt.xticks(rotation=45)
        plt.grid(axis='y', linestyle='--', alpha=0.5)
        plt.tight_layout()
        plt.show()

    widgets.interact(plot_upto, index=widgets.IntSlider(min=1, max=len(times), step=1, value=8))

def interactive_ui():
    city_input = widgets.Text(value='', placeholder='Enter city (e.g. Paris)', description='City:', layout=widgets.Layout(width='300px'))
    question_input = widgets.Text(value='', placeholder='e.g. Will it rain in Paris tomorrow?', description='Ask:', layout=widgets.Layout(width='500px'))
    source_dropdown = widgets.Dropdown(options=['openweathermap', 'mocksource'], value='openweathermap', description='Source:', layout=widgets.Layout(width='200px'))

    output_area = widgets.Output(layout=widgets.Layout(
    border='1px solid #e0e0e0',
    padding='22px',
    max_height='440px',
    overflow='auto',
    background_color='#f7fafc',
    border_radius='12px'
))


    forecast_btn = widgets.Button(description="📊 Forecast Charts", button_style='info', tooltip='Show temperature and precipitation charts')
    question_btn = widgets.Button(description="🧠 Answer", button_style='success', tooltip='Get AI-style weather answer')
    clear_btn = widgets.Button(description="🧹 Clear", button_style='warning', tooltip='Clear inputs and output')

    def toggle_buttons(*args):
        forecast_btn.disabled = not bool(city_input.value.strip())
        question_btn.disabled = not bool(question_input.value.strip())

    city_input.observe(toggle_buttons, 'value')
    question_input.observe(toggle_buttons, 'value')
    toggle_buttons()

    def on_forecast_button_click(_):
        with output_area:
            clear_output()
            city = city_input.value.strip()
            if not city:
                print("❌ Please enter a city.")
                return
            try:
                data = fetch_weather_data(city, source=source_dropdown.value)
                interactive_temperature_plot(data, city)
                interactive_precipitation_plot(data, city)
            except Exception as e:
                print(f"❌ Error: {e}")

    def on_question_button_click(_):
        with output_area:
            clear_output()
            question = question_input.value.strip()
            if not question:
                print("❌ Please enter a question.")
                return
            parsed = parse_weather_question(question)
            response = generate_weather_response(parsed, source=source_dropdown.value)
            print(response)

    def on_clear_button_click(_):
        city_input.value = ''
        question_input.value = ''
        with output_area:
            clear_output()

    forecast_btn.on_click(on_forecast_button_click)
    question_btn.on_click(on_question_button_click)
    clear_btn.on_click(on_clear_button_click)

    input_row = widgets.HBox([city_input, source_dropdown, forecast_btn, clear_btn], layout=widgets.Layout(margin='10px 0'))
    question_row = widgets.HBox([question_input, question_btn], layout=widgets.Layout(margin='10px 0'))

    header = widgets.HTML("""

        🌤️ Interactive Weather Assistant

""")


    main_box = widgets.VBox([
    header,
    input_row,
    question_row,
    output_area
], layout=widgets.Layout(
    width='820px',
    max_width='96vw',
    margin='48px auto 48px auto',
    padding='36px 48px 40px 48px',
    background_color='#fff',
    border_radius='26px',
    border='1.5px solid #e3e8ee',
    box_shadow='0 8px 32px rgba(44, 62, 80, 0.13)',
    display='flex',
    flex_flow='column',
    align_items='center',
    justify_content='center',
    font_family='Segoe UI, Arial, sans-serif'
))


    display(main_box)

interactive_ui()

## 🧪 Testing and Examples

In [ ]:
# Include sample input/output for each function

#-------------------------------------------------------------------------------------------------------
# 1.🌤️ Weather Data Functions
#-------------------------------------------------------------------------------------------------------

# 1.1 ✅ Define the get_weather_data() Function
from typing import Union, List, Dict

def get_weather_data(location: Union[str, List[str]], forecast_days: int = 5) -> Dict[str, dict]:
    """
    Fetch weather data for one or more locations and return it as a dictionary.

    Args:
        location (str or list): A single location name or a list of locations.
        forecast_days (int): Number of forecast days to retrieve (default is 5).

    Returns:
        dict: A dictionary mapping each location to its weather data or error.
    """
    if isinstance(location, str):
        location = [location]

    results = {}

    for loc in location:
        try:
            weather_response = get_weather(loc)
            if not hasattr(weather_response, 'dict'):
                raise TypeError("Weather object does not support dict() conversion.")

            weather_data = weather_response.dict()
            forecast = weather_data.get("forecast", [])
            current = weather_data.get("current_conditions", {})

            if not forecast:
                raise ValueError("Forecast data is missing or empty.")

            results[loc] = {
                "current_conditions": current or "No current data available",
                "forecast": forecast[:forecast_days]
            }

        except Exception as e:
            results[loc] = {"error": f"Failed to retrieve weather for {loc}: {e}"}

    return results

# 1.2 ✅ Run doctest

import doctest
doctest.testmod()


def get_weather_data(location: Union[str, List[str]], forecast_days: int = 5) -> Dict[str, dict]:
    """
    >>> isinstance(get_weather_data("Paris"), dict)
    True
    >>> "error" in get_weather_data("FakeCity")["FakeCity"]
    True
    """

# 1.3 ✅ Run unittest

import unittest

class MockWeatherResponse:
    def __init__(self, valid=True):
        self.valid = valid

    def dict(self):
        if self.valid:
            return {
                "current_conditions": {"temperature": 20},
                "forecast": [{"day": 1}, {"day": 2}, {"day": 3}]
            }
        else:
            return {}

def get_weather(location):
    if location == "FakeCity":
        raise ValueError("Location not found")
    return MockWeatherResponse()

class TestGetWeatherData(unittest.TestCase):
    def test_valid_city(self):
        result = get_weather_data("Paris", forecast_days=2)
        self.assertIn("current_conditions", result["Paris"])
        self.assertEqual(len(result["Paris"]["forecast"]), 2)

    def test_invalid_city(self):
        result = get_weather_data("FakeCity")
        self.assertIn("error", result["FakeCity"])

unittest.main(argv=[''], verbosity=2, exit=False)

# 1.4 🐞 Debugging

def debug_locations(locations):
    print("🌍 Fetching weather for multiple locations...")
    results = get_weather_data(locations)
    for loc, data in results.items():
        print(f"\n📍 Location: {loc}")
        if "error" in data:
            print(f"❌ Error: {data['error']}")
        else:
            print(f"✅ Current: {data['current_conditions']}")
            print(f"📅 Forecast ({len(data['forecast'])} days):")
            for day in data['forecast']:
                print(day)

debug_locations(["London", "FakeCity", "Berlin"])

#-------------------------------------------------------------------------------------------------------
# 2. 📊 Visualisation Functions
#-------------------------------------------------------------------------------------------------------

# 2.1 Define the Visualisation Functions

# 2.1.1 Temperature Line Chart (Plotly)

import plotly.express as px
import pandas as pd

def create_temperature_visualisation_interactive(weather_data):
    """
    Create an interactive temperature line chart using Plotly.
    """
    rows = []
    for location, times_data in weather_data.items():
        for time, info in times_data.items():
            rows.append({
                "Location": location,
                "Time": time,
                "Temperature": info["temperature"]
            })

    df = pd.DataFrame(rows)
    fig = px.line(
        df, x="Time", y="Temperature", color="Location",
        markers=True, title="Interactive Temperature Forecast",
        labels={"Temperature": "Temp (°C)", "Time": "Time Period"}
    )
    fig.update_traces(mode='lines+markers', hovertemplate="%{y}°C at %{x}<br>%{fullData.name}")
    fig.update_layout(xaxis_tickangle=-45)
    fig.show()

# 2.1.2 Precipitation Bar Chart (Matplotlib)

import pandas as pd
import matplotlib.pyplot as plt

def create_precipitation_visualisation_from_df(df: pd.DataFrame, output_type='display'):
    """
    Create a grouped bar chart of precipitation chances using a pandas DataFrame.
    """
    plt.style.use('seaborn-darkgrid')
    fig, ax = plt.subplots(figsize=(10, 6))

    locations = df['Location'].unique()
    time_labels = df['Time'].unique()
    width = 0.8 / len(locations)
    x = range(len(time_labels))

    for idx, loc in enumerate(locations):
        loc_data = df[df['Location'] == loc]
        loc_data = loc_data.set_index('Time').reindex(time_labels).reset_index()
        offsets = [i + (idx * width) for i in x]
        precip = loc_data['Precipitation'].fillna(0).tolist()

        ax.bar(offsets, precip, width=width, label=loc)
        for i, value in enumerate(precip):
            ax.text(offsets[i], value + 1, f"{value}%", ha='center', fontsize=8)

    ax.set(
        title="Chance of Rain by Location",
        xlabel="Time Period",
        ylabel="Chance of Rain (%)"
    )
    ax.set_xticks([i + width * (len(locations)-1)/2 for i in x])
    ax.set_xticklabels(time_labels, rotation=45)
    ax.legend()
    ax.grid(True)

    if output_type == 'figure':
        return fig
    plt.show()

# 2.2 ✅ Run doctest for Format Check

import doctest
doctest.testmod()


def create_temperature_visualisation_interactive(weather_data):
    """
    >>> isinstance(weather_data, dict)
    True
    """
    ...

# 2.3 ✅ Run unittest for Chart Functions

import unittest

class TestVisualisationData(unittest.TestCase):
    def test_dataframe_conversion(self):
        data = {
            "London": {"Mon": {"temperature": 21}, "Tue": {"temperature": 22}},
            "Paris": {"Mon": {"temperature": 19}, "Tue": {"temperature": 20}},
        }
        rows = []
        for location, times_data in data.items():
            for time, info in times_data.items():
                rows.append({
                    "Location": location,
                    "Time": time,
                    "Temperature": info["temperature"]
                })
        df = pd.DataFrame(rows)
        self.assertEqual(len(df), 4)
        self.assertIn("Location", df.columns)

unittest.main(argv=[''], verbosity=2, exit=False)

# 2.4 ✅ Run assert Tests (Pytest Style)

sample_data = {
    "London": {"Monday": {"temperature": 20}, "Tuesday": {"temperature": 22}},
    "Paris": {"Monday": {"temperature": 18}, "Tuesday": {"temperature": 21}}
}

def test_temperature_data():
    rows = []
    for location, times_data in sample_data.items():
        for time, info in times_data.items():
            rows.append({
                "Location": location,
                "Time": time,
                "Temperature": info["temperature"]
            })
    df = pd.DataFrame(rows)
    assert df.shape[0] == 4
    assert "Temperature" in df.columns

test_temperature_data()
print("All visualisation data tests passed.")

# 2.5 🐞 Debug with Sample Charts

# Temperature (Interactive)
create_temperature_visualisation_interactive({
    "London": {"Monday": {"temperature": 20}, "Tuesday": {"temperature": 22}},
    "Paris": {"Monday": {"temperature": 18}, "Tuesday": {"temperature": 21}}
})

# Precipitation (Grouped Bar)
df = pd.DataFrame({
    "Location": ["London", "Paris", "London", "Paris"],
    "Time": ["Monday", "Monday", "Tuesday", "Tuesday"],
    "Precipitation": [30, 50, 10, 60]
})
create_precipitation_visualisation_from_df(df)

#-------------------------------------------------------------------------------------------------------
# 3. 🤖 Natural Language Processing
#-------------------------------------------------------------------------------------------------------

# 3.1 ✅ Define generate_weather_response()

def generate_weather_response(parsed, data, temp_unit='Celsius', wind_unit='km/h'):
    loc = parsed['location']
    time = parsed['time_period']
    attr = parsed['attribute']

    if loc not in data or time not in data[loc]:
        return f"❌ No forecast available for {loc} {time}."

    forecast = data[loc][time]

    def convert_temp(c): return f"{convert_temperature(c, temp_unit)}°{'F' if temp_unit == 'Fahrenheit' else 'C'}"
    def convert_wind(w): return f"{convert_wind_speed(w, wind_unit)} {wind_unit}"

    responses = {
        'temperature': lambda: f"🌡️ Temperature in {loc} {time} is {convert_temp(forecast.get('temperature', 'unknown'))}.",
        'rain': lambda: f"🌧️ Expect {'rain' if forecast.get('rain') else 'no rain'} in {loc} {time}.",
        'snow': lambda: f"❄️ Expect {'snow' if forecast.get('snow') else 'no snow'} in {loc} {time}.",
        'wind': lambda: f"💨 Wind speed in {loc} {time} is {convert_wind(forecast.get('wind', 'unknown'))}.",
        'humidity': lambda: f"💧 Humidity in {loc} {time} is {forecast.get('humidity', 'unknown')}%."
    }

    if attr in responses:
        return responses[attr]()
    else:
        temp = convert_temp(forecast.get('temperature', 'unknown'))
        rain = 'rain' if forecast.get('rain') else 'no rain'
        return f"In {loc} {time}, temperature is {temp} with {rain}."

# 3.2 ✅ Add doctest

def generate_weather_response(parsed, data, temp_unit='Celsius', wind_unit='km/h'):
    """
    >>> parsed = {'location': 'Paris', 'time_period': 'today', 'attribute': 'temperature'}
    >>> data = {'Paris': {'today': {'temperature': 20}}}
    >>> generate_weather_response(parsed, data)
    '🌡️ Temperature in Paris today is 20°C.'
    """

import doctest
doctest.testmod()

# 3.3 ✅ Unit Testing with unittest

# Mock functions for temperature/wind conversion:

def convert_temperature(value, unit):
    return round((value * 9/5 + 32) if unit == 'Fahrenheit' else value)

def convert_wind_speed(value, unit):
    return round((value / 1.609) if unit == 'mph' else value)

# Then test:
import unittest

class TestGenerateWeatherResponse(unittest.TestCase):
    def test_temperature_celsius(self):
        parsed = {'location': 'Paris', 'time_period': 'today', 'attribute': 'temperature'}
        data = {'Paris': {'today': {'temperature': 25}}}
        result = generate_weather_response(parsed, data)
        self.assertIn("25°C", result)

    def test_missing_location(self):
        parsed = {'location': 'Mars', 'time_period': 'today', 'attribute': 'temperature'}
        data = {'Paris': {'today': {'temperature': 25}}}
        result = generate_weather_response(parsed, data)
        self.assertIn("❌", result)

unittest.main(argv=[''], verbosity=2, exit=False)

# 3.4 ✅ Use assert (Pytest-style tests)

parsed = {'location': 'Paris', 'time_period': 'today', 'attribute': 'wind'}
data = {'Paris': {'today': {'wind': 20}}}

def test_weather_response():
    result = generate_weather_response(parsed, data, wind_unit='km/h')
    assert "20 km/h" in result
    assert "💨" in result

test_weather_response()
print("All pytest-style tests passed.")

# 3.5 🐞 Debug with Sample Data

parsed_inputs = [
    {'location': 'London', 'time_period': 'today', 'attribute': 'temperature'},
    {'location': 'London', 'time_period': 'today', 'attribute': 'rain'},
    {'location': 'London', 'time_period': 'today', 'attribute': 'wind'},
    {'location': 'London', 'time_period': 'today', 'attribute': 'snow'},
    {'location': 'London', 'time_period': 'today', 'attribute': 'general'},
]

weather_data = {
    "London": {
        "today": {
            "temperature": 22,
            "rain": False,
            "snow": False,
            "wind": 15,
            "humidity": 70
        }
    }
}

for parsed in parsed_inputs:
    print(generate_weather_response(parsed, weather_data))

#-------------------------------------------------------------------------------------------------------
# 4. 🧭 User Interface
#-------------------------------------------------------------------------------------------------------
# 4.1 ✅ Define the Menu Interface Functions

import pyinputplus as pyip

def print_banner():
    print("\n" + "="*40)
    print("☁️  Welcome to the Weather Assistant!  ☁️")
    print("="*40)

def main_menu():
    print("\n=== 🌦️ Weather Assistant Menu ===")
    return pyip.inputMenu(
        ["🌤️  Ask Weather Question", "📊  Visualize Weather Data", "🚪  Exit"],
        lettered=True,
        numbered=False,
        prompt="\n🌈 What would you like to do?\n",
        blank=False,
        limit=3,
        retries=2,
        default="🚪  Exit"
    )

def visualization_menu():
    return pyip.inputMenu(
        ["📈  Temperature", "🌧️  Precipitation", "🔙  Back to Main Menu"],
        lettered=True,
        numbered=False,
        prompt="\n📊 Choose a visualization type:\n",
        blank=False,
        limit=3,
        retries=2,
        default="🔙  Back to Main Menu"
    )

def run_interface():
    print_banner()
    while True:
        choice = main_menu()
        if "Ask Weather" in choice:
            print("🗣️  You chose to ask a weather question.")
        elif "Visualize" in choice:
            while True:
                sub_choice = visualization_menu()
                if "Temperature" in sub_choice:
                    print("📈 Showing temperature chart...")
                elif "Precipitation" in sub_choice:
                    print("🌧️ Showing precipitation chart...")
                elif "Back" in sub_choice:
                    break
        elif "Exit" in choice:
            print("👋 Exiting Weather Assistant. Stay safe!")
            break

# 4.2 ✅ Add Doctests

def main_menu():
    """
    >>> isinstance(main_menu(), str)
    True
    """

import doctest
doctest.testmod()

# 4.3 ✅ Unittest for Menu Logic

import unittest

class TestMenuStructure(unittest.TestCase):
    def test_main_menu_options(self):
        options = [
            "🌤️  Ask Weather Question",
            "📊  Visualize Weather Data",
            "🚪  Exit"
        ]
        self.assertEqual(len(options), 3)
        self.assertIn("🌤️  Ask Weather Question", options)

unittest.main(argv=[''], verbosity=2, exit=False)


# 4.4 ✅ Assert-Based Tests (Pytest-style)

def test_menu_choice_return_type():
    menu_response = "🌧️  Precipitation"
    assert isinstance(menu_response, str)
    assert "Precipitation" in menu_response

test_menu_choice_return_type()
print("✅ Menu tests passed.")

# 4.5 🐞 Debug with Manual Simulation

# Run this in a local terminal or Colab (with ipywidgets disabled):
run_interface()

# Expected steps:
# - Shows banner
# - Main menu with 3 choices
# - If 'Visualize Weather Data' is chosen:
#     - Shows 3 submenu options
#     - Returns to main menu on 'Back'


